In [ ]:
import os
import cv2
import numpy as np

# Define folder paths for original files and mask files
original_folder = '/Users/zhanghan/Desktop/Lab/LiangLab/Data/Recording_chromatophore activity/Alice/20250521/frame_extract'  # Original image folder, note: convert to PNG for subsequent operations
mask_folder = '/Users/zhanghan/Desktop/Lab/LiangLab/Data/Recording_chromatophore activity/Alice/20250521/mask'         # Mask image folder
output_folder = '/Users/zhanghan/Desktop/Lab/LiangLab/Data/Recording_chromatophore activity/Alice/20250521/mask_seg'     # Output folder for segmentation results

# Ensure output folder exists, create if not
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Get all filenames in the folders
original_files = sorted(os.listdir(original_folder))
mask_files = sorted(os.listdir(mask_folder))

# Iterate through files and process
for original_file, mask_file in zip(original_files, mask_files):
    # Construct complete file paths
    original_path = os.path.join(original_folder, original_file)
    mask_path = os.path.join(mask_folder, mask_file)

    # Read original image and mask image
    original_image = cv2.imread(original_path)
    mask_image = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # Read mask in grayscale mode

    # Check if original image and mask dimensions match
    if original_image.shape[:2] != mask_image.shape:
        print(f"Error: Image and mask dimensions do not match for {original_file}")
        continue

    # Find all connected components in the mask
    contours, hierarchy = cv2.findContours(mask_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Skip current image if no contours are found
    if len(contours) == 0:
        print(f"Warning: No contours found in mask for {original_file}")
        continue

    # Calculate area of each connected component and find the largest one
    max_area = 0
    max_contour = None
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > max_area:
            max_area = area
            max_contour = contour

    # Create a new mask retaining only the largest region
    largest_mask = np.zeros_like(mask_image)
    cv2.drawContours(largest_mask, [max_contour], -1, 255, thickness=cv2.FILLED)

    # Segment original image using the new mask
    segmented_image = cv2.bitwise_and(original_image, original_image, mask=largest_mask)

    # Construct output file path
    output_path = os.path.join(output_folder, original_file)

    # Save segmented image
    cv2.imwrite(output_path, segmented_image)